In [1]:
from satpy import Scene, MultiScene, find_files_and_readers
from satpy.composites import GenericCompositor
from pyresample.geometry import AreaDefinition
from glob import glob
import dask
import os

from shapely.geometry import Point, Polygon

In [2]:
os.environ['SATPY_DASK_CHUNK_SIZE'] = '32MiB'
dask.config.set(num_workers=4)

In [3]:
dlist = range(1, 5)

In [4]:
scn_list = []
for path in dlist:
    files = find_files_and_readers(base_dir=f'C:\\Arbetsmapp\\ALGOTL\\{path}', reader='msi_safe')
    scn = Scene(filenames=files, reader='msi_safe')
    scn_list.append(scn)
scn_list

In [5]:
print(scn_list[1])

In [6]:
algotl_10 = AreaDefinition.from_area_of_interest(
    area_id="algotltest_full", projection='epsg:3006', 
    shape=[20000, 18000], center=[720573.556, 6371774.759], 
    resolution=[10, 10]
    )

In [7]:
# 20000x18000, center=[720573.556, 6371774.759]
c_x = 720573.556
c_y = 6371774.759
ll_x = 630573.556
ll_y = 6271774.759
ur_x = 810573.556
ur_y = 6471774.759

c = Point(c_x, c_y)
ll = Point((ll_x, ll_y))
ul = Point((ll_x, ur_y))
lr = Point((ur_x, ll_y))
ur = Point((ur_x, ur_y))

In [8]:
overlap = 0.1
quad_width = 10000
quad_height = 9000
ol_x = quad_width * overlap
ol_y = quad_height * overlap
quad_x_ol = quad_width + ol_x
quad_y_ol = quad_height + ol_y

# p_ll_c = Point((ll_x+(c_x+ol))/2, (ll_y+(c_y+ol))/2)
# p_ul_c = Point((ll_x+(c_x+ol))/2, (ur_y+(c_y-ol))/2)
# p_lr_c = Point((ur_x+(c_x-ol))/2, (ll_y+(c_y+ol))/2)
# p_ur_c = Point((ur_x+(c_x-ol))/2, (ur_y+(c_y-ol))/2)
# print('GEOMETRYCOLLECTION(' + str(ll_c) + ',', str(ul_c) + ',', str(lr_c) + ',', str(ur_c) + ')')
# GEOMETRYCOLLECTION(POINT (675573.556 6321774.759), POINT (675573.556 6421774.759), POINT (765573.556 6321774.759), POINT (765573.556 6421774.759))
# GEOMETRYCOLLECTION(POINT (676073.556 6322274.759), POINT (676073.556 6421274.759), POIN1T (765073.556 6322274.759), POINT (765073.556 6421274.759))


ll_c = [(ll_x+(c_x+ol_x))/2, (ll_y+(c_y+ol_y))/2]
ul_c = [(ll_x+(c_x+ol_x))/2, (ur_y+(c_y-ol_y))/2]
lr_c = [(ur_x+(c_x-ol_x))/2, (ll_y+(c_y+ol_y))/2]
ur_c = [(ur_x+(c_x-ol_x))/2, (ur_y+(c_y-ol_y))/2]

center_points = {'ll': ll_c, 'ul': ul_c, 'lr': lr_c, 'ur': ur_c}

In [9]:
algotl_quadrants = []
for quadrant, center in center_points.items():
    algotl_bbox = AreaDefinition.from_area_of_interest(area_id=f"algotl_{quadrant}", projection='epsg:3006', shape=[quad_x_ol, quad_y_ol], center=center, resolution=[10, 10])
    algotl_quadrants.append(algotl_bbox)

In [10]:
algotl_quadrants

[Area ID: algotl_ll
 Description: algotl_ll
 Projection: {'ellps': 'GRS80', 'no_defs': 'None', 'proj': 'utm', 'type': 'crs', 'units': 'm', 'zone': '33'}
 Number of columns: 9900
 Number of rows: 11000
 Area extent: (626573.556, 6267224.759, 725573.556, 6377224.759),
 Area ID: algotl_ul
 Description: algotl_ul
 Projection: {'ellps': 'GRS80', 'no_defs': 'None', 'proj': 'utm', 'type': 'crs', 'units': 'm', 'zone': '33'}
 Number of columns: 9900
 Number of rows: 11000
 Area extent: (626573.556, 6366324.759, 725573.556, 6476324.759),
 Area ID: algotl_lr
 Description: algotl_lr
 Projection: {'ellps': 'GRS80', 'no_defs': 'None', 'proj': 'utm', 'type': 'crs', 'units': 'm', 'zone': '33'}
 Number of columns: 9900
 Number of rows: 11000
 Area extent: (715573.556, 6267224.759, 814573.556, 6377224.759),
 Area ID: algotl_ur
 Description: algotl_ur
 Projection: {'ellps': 'GRS80', 'no_defs': 'None', 'proj': 'utm', 'type': 'crs', 'units': 'm', 'zone': '33'}
 Number of columns: 9900
 Number of rows: 1100

In [11]:
algotl_10.crs

<Projected CRS: EPSG:3006>
Name: SWEREF99 TM
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Sweden - onshore and offshore.
- bounds: (10.03, 54.96, 24.17, 69.07)
Coordinate Operation:
- name: SWEREF99 TM
- method: Transverse Mercator
Datum: SWEREF99
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
composite = 'true_color'

In [13]:
mscn = MultiScene(scn_list)
mscn.load(['true_color'])
algotl_mscn = mscn.resample(algotl_quadrants[2])
algotl_scn = algotl_mscn.blend()

c:\Users\k000851\AppData\Local\miniforge3\envs\ALGOTLvisTest\Lib\site-packages\satpy\readers\msi_safe.py:321: RuntimeWarning: Mean of empty slice
  angles = np.nanmean(np.dstack(arrays), -1)
c:\Users\k000851\AppData\Local\miniforge3\envs\ALGOTLvisTest\Lib\site-packages\satpy\readers\msi_safe.py:321: RuntimeWarning: Mean of empty slice
  angles = np.nanmean(np.dstack(arrays), -1)
c:\Users\k000851\AppData\Local\miniforge3\envs\ALGOTLvisTest\Lib\site-packages\pyresample\kd_tree.py:1153: PerformanceWarning: Increasing number of chunks by factor of 13
  res = blockwise(_my_index, dst_adims,
c:\Users\k000851\AppData\Local\miniforge3\envs\ALGOTLvisTest\Lib\site-packages\satpy\readers\msi_safe.py:321: RuntimeWarning: Mean of empty slice
  angles = np.nanmean(np.dstack(arrays), -1)
c:\Users\k000851\AppData\Local\miniforge3\envs\ALGOTLvisTest\Lib\site-packages\satpy\readers\msi_safe.py:321: RuntimeWarning: Mean of empty slice
  angles = np.nanmean(np.dstack(arrays), -1)
c:\Users\k000851\AppData\

In [14]:
algotl_scn.save_dataset(composite, 'S2A_240727_lr_10_tc_3006.tif')

c:\Users\k000851\AppData\Local\miniforge3\envs\ALGOTLvisTest\Lib\site-packages\dask\core.py:127: RuntimeWarning: invalid value encountered in log
  return func(*(_execute_task(a, cache) for a in args))


In [15]:
# small_scn = algotl_scn.resample(algotl_scn.finest_area().copy(height=2000, width=1800), resampler="nearest")

In [16]:
# small_scn.save_dataset(composite, 'sentinel2_msi_algotl_small01.tif')